<h1>Overpass API Notebook</h1>

<p>This notebook is dedicated to call the overpass API and extract data for our ADS Project. The data will enrich our rental price prediction models with relevant geo-spacial data_ in the vicitnity of a given apartment such as restaurants, supermarkets and other points of interest.<p> 

In [42]:
import json
import requests
import pandas as pd


# Request all restaurants, cafes, and supermarkets in the Kanton of Zurich, Switzerland
overpass_url = "http://overpass-api.de/api/interpreter"
overpass_query = """
[out:json];
area["alt_name"="Kanton Zürich"]->.a;
(node["amenity"="restaurant"](area.a);
 node["amenity"="cafe"](area.a);
 node["shop"="supermarket"](area.a););
out center;
"""


# Server response
response = requests.get(overpass_url, params={'data': overpass_query})

data = response.json()

# Extract relevant data from the JSON response
places = []
for element in data['elements']:
    if 'tags' in element:
        place_type = element['tags'].get('amenity') or element['tags'].get('shop')
        places.append({
            'id': element['id'],
            'type': place_type,
            'lat': element['lat'],
            'lon': element['lon'],
            'name': element['tags'].get('name', ''),
            'brand': element['tags'].get('brand', ''),
            'address_city': element['tags'].get('addr:city', ''),
            'address_zip': element['tags'].get('addr:postcode', ''),
            'address_street': element['tags'].get('addr:street', ''),
            'address_number': element['tags'].get('addr:housenumber', ''),
            'website': element['tags'].get('website', '')
            

        })

# Convert the list of dictionaries to a pandas DataFrame
places_df = pd.DataFrame(places)

# Display the DataFrame
places_df


,id,type,lat,lon,name,brand,address_city,address_zip,address_street,address_number,website
0,34039212,restaurant,47.495518,8.741556,Hallenbadrestaurant Geiselweid,,,,,,
1,34040796,restaurant,47.483065,8.705704,Schwimmbadrestaurant Töss,,,,,,
2,39947904,supermarket,47.376732,8.542161,Coop,Coop,Zürich,8001,Bahnhofbrücke,1,
3,48932835,supermarket,47.375020,8.522895,Migros,Migros,Zürich,8004,Wengistrasse,7,https://filialen.migros.ch/de/migros-supermark...
4,57370928,restaurant,47.245360,8.726042,Metzg,,,,,,
...,...,...,...,...,...,...,...,...,...,...,...
3563,11744871126,restaurant,47.363958,8.547829,Kaisin,,,,Dufourstrasse,23,https://kaisin.ch/
3564,11746067526,restaurant,47.409336,8.547044,Pho Bao,,,,,,
3565,11748556157,cafe,47.393331,8.530811,LOKAL,,,,,,
3566,11749827343,cafe,47.337672,8.575697,Bäckerei Hausammann,,Zollikon,8702,Alte Landstrasse,91,https://www.zopfbeck.ch/fil.zol.html


In [47]:
types = ['restaurant', 'cafe', 'supermarket']

for i in types:
    print(i ,':', (places_df['type'] == i).sum())

restaurant : 2341
cafe : 601
supermarket : 624
